Please note that this script may not always work as expected due to certain limitations and potential errors. Firstly, Alpha Vantage, the API we're using, imposes a limit on the number of API calls that can be made within a certain period. For free users, this limit is typically set at 5 API requests per minute and 500 requests per day. If you exceed this limit, the API may not return the expected data, which can cause the script to fail or produce inaccurate results.

Secondly, sometimes an API might return an error, or the structure of the API response might change unexpectedly. Our script expects certain keys, such as 'SharesOutstanding', to be present in the API's response. If these keys are not included in the response due to an error or change on the API's side, a KeyError will be triggered, causing the script to stop. It's important to be aware of these potential issues when running the script and interpreting its output.

In [98]:
import requests
import numpy as np

# Constants
DISCOUNT_RATE = 0.1
TERMINAL_GROWTH_RATE = 0.02  # commonly used rate for perpetuity
API_KEY = 'Alpha Vantage API'
SYMBOL = 'AAPL'

# Get the cash flow statement
cash_flow_url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={SYMBOL}&apikey={API_KEY}'
cash_flow_data = requests.get(cash_flow_url).json()

try:
    cash_flow_statement = cash_flow_data['annualReports']
except KeyError:
    print("Error: Unable to find 'annualReports' in the API response")
    cash_flow_statement = []

# Get the free cash flow for the last fiscal year
if cash_flow_statement:
    try:
        free_cash_flow_last_year = float(cash_flow_statement[0]['operatingCashflow']) - float(cash_flow_statement[0]['capitalExpenditures'])
    except KeyError:
        print("Error: Unable to find 'operatingCashflow' or 'capitalExpenditures' in the API response")
        free_cash_flow_last_year = 0
else:
    print("Error: No data available in 'annualReports'")
    free_cash_flow_last_year = 0

# Estimate the growth rate of the free cash flow using CAGR formula
cash_flows = [float(report.get('operatingCashflow', 0)) - float(report.get('capitalExpenditures', 0)) for report in cash_flow_statement]
years = len(cash_flows)
if cash_flows and cash_flows[-1] != 0:  # add check for cash_flows list being not empty
    growth_rate = (cash_flows[0] / cash_flows[-1]) ** (1 / years) - 1
else:
    print("Error: No cash flow data available or last year's cash flow is zero")
    growth_rate = 0

# Calculate the present value of future cash flows for the first 5 years
years = np.arange(1, 6)
cash_flows = free_cash_flow_last_year * (1 + growth_rate) ** years
discounted_cash_flows = cash_flows / (1 + DISCOUNT_RATE) ** years

# Calculate the terminal value and its present value
terminal_value = (cash_flows[-1] * (1 + TERMINAL_GROWTH_RATE)) / (DISCOUNT_RATE - TERMINAL_GROWTH_RATE)
discounted_terminal_value = terminal_value / (1 + DISCOUNT_RATE) ** years[-1]

# Add the discounted terminal value to the discounted cash flows
discounted_cash_flows = np.append(discounted_cash_flows, discounted_terminal_value)

# Get the balance sheet data
balance_sheet_url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={SYMBOL}&apikey={API_KEY}'
balance_sheet_data = requests.get(balance_sheet_url).json()

try:
    balance_sheet = balance_sheet_data['annualReports']
except KeyError:
    print("Error: Unable to find 'annualReports' in the API response")
    balance_sheet = []

# Get the total debt, cash and equivalents
total_debt = float(balance_sheet[0].get('shortLongTermDebtTotal', 0))
cash_and_equivalents = float(balance_sheet[0].get('cashAndCashEquivalentsAtCarryingValue', 0))

# Get the number of shares outstanding
overview_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={SYMBOL}&apikey={API_KEY}'
overview_data = requests.get(overview_url).json()

try:
    shares_outstanding = float(overview_data['SharesOutstanding'])
except KeyError:
    print("Error: Unable to find 'SharesOutstanding' in the API response")
    shares_outstanding = 1  # You might want to handle this situation differently

# Calculate and print the intrinsic value of the stock
intrinsic_value = (np.sum(discounted_cash_flows) - total_debt + cash_and_equivalents) / shares_outstanding
print(f'The intrinsic value of the {SYMBOL} stock is: {intrinsic_value}')

The intrinsic value of the AAPL stock is: 121.25555994785546
